In [13]:
%pip install matplotlib==3.8.2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from sklearn.model_selection import train_test_split
import sklearn.model_selection as model_selection
import matplotlib.pyplot as plt
import math
import keras as k
import pandas as pd
import numpy as np
from copy import copy
import scipy.io
from sklearn.decomposition import PCA

from scipy.signal import find_peaks
from scipy import interpolate
from numpy import sin,cos,pi,array,linspace,cumsum,asarray,dot,ones
from pylab import plot, legend, axis, show, randint, randn, std,lstsq

from sklearn.manifold import MDS
import csv
import os
from tqdm import tqdm

ERROR: Could not find a version that satisfies the requirement matplotlib==3.8.2 (from versions: 0.86, 0.86.1, 0.86.2, 0.91.0, 0.91.1, 1.0.1, 1.1.0, 1.1.1, 1.2.0, 1.2.1, 1.3.0, 1.3.1, 1.4.0, 1.4.1rc1, 1.4.1, 1.4.2, 1.4.3, 1.5.0, 1.5.1, 1.5.2, 1.5.3, 2.0.0b1, 2.0.0b2, 2.0.0b3, 2.0.0b4, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.0.2, 2.1.0rc1, 2.1.0, 2.1.1, 2.1.2, 2.2.0rc1, 2.2.0, 2.2.2, 2.2.3, 2.2.4, 2.2.5, 3.0.0rc2, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.1.0rc1, 3.1.0rc2, 3.1.0, 3.1.1, 3.1.2, 3.1.3, 3.2.0rc1, 3.2.0rc3, 3.2.0, 3.2.1, 3.2.2, 3.3.0rc1, 3.3.0, 3.3.1, 3.3.2, 3.3.3, 3.3.4, 3.4.0rc1, 3.4.0rc2, 3.4.0rc3, 3.4.0, 3.4.1, 3.4.2, 3.4.3, 3.5.0b1, 3.5.0rc1, 3.5.0, 3.5.1, 3.5.2, 3.5.3, 3.6.0rc1, 3.6.0rc2, 3.6.0, 3.6.1, 3.6.2)
ERROR: No matching distribution found for matplotlib==3.8.2
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Print all versions
print(tf.__version__)
print(k.__version__)
print(np.__version__)

2.8.2
2.8.0
1.23.2


In [15]:
#Load pathway where fourierseries, util, phaser, dataloader, data are stored
%cd /home/victorphilippecodex/Documents/Hand_Signature/WholeGroup1/Python_Inputs

/home/victorphilippecodex/Documents/Hand_Signature/WholeGroup1/Python_Inputs


In [16]:
!ls -l fourierseries.py

-rw-rw-r-- 1 victorphilippecodex victorphilippecodex 7259 Aug 17 10:21 fourierseries.py


In [17]:
import fourierseries
import util
import phaser
import dataloader

In [18]:
def find_phase(k):
    """
    Detrend and compute the phase estimate using Phaser
    INPUT:
      k -- dataframe
    OUTPUT:
      k -- dataframe
    """
    #l = ['hip_flexion_l','hip_flexion_r'] # Phase variables = hip flexion angles
    y = np.array(k)
    print(y.shape)
    y = util.detrend(y.T).T
    print(y.shape)
    phsr = phaser.Phaser(y=y)
    k[:] = phsr.phaserEval(y)[0,:]
    return k

In [19]:
def vonMies(t,t_0, b):
    out = np.exp(b*np.cos(t-t_0))/(2*pi*iv(0, b))
    return out

In [20]:
path = '/home/victorphilippecodex/Documents/Hand_Signature/WholeGroup1'

In [21]:
# Non-changing variables 

# number of trials in dataset 
trialnum = 252 # 72 total trials

# number of samples in each trial
trialsamp = 2100

# number of features collected per trial
feats = 15

#Batch size - same as the number of traintrials
batch_size = trialnum

# Number of Layers
numlayers = 1

# Choose the number of iterations to train the model- if this script has been run previously enter a value greater than was 
# inputted before and rerun the script. 
finalepoch = 10000

# load the input data/kinematics
datafilepath = '/home/victorphilippecodex/Documents/Hand_Signature/WholeGroup1/DownsampledGroup1.csv' #input data
all_csvnp = np.loadtxt(datafilepath,delimiter=',').T

# reshape all the input data into a tensor
all_inputdata_s = all_csvnp.reshape(trialnum,trialsamp,feats) 
csvnp = all_inputdata_s
print('original input data shape is: '+ str(all_csvnp.shape ))
print('input data reshaped is: '+ str(all_inputdata_s.shape))

original input data shape is: (15, 529200)
input data reshaped is: (252, 2100, 15)


In [22]:
test_model_nodes = [256] 
seqs = [699] #lookback parameter

# run multiple model architechtures many times to test stability of cost function outputs
runs = 1 # stability analysis - repeat each model architecture this many times
test_model_seq = np.repeat(seqs, runs)

count = np.arange(runs)

All_nodes = np.empty([0,1], dtype='int')
All_seq = np.empty([0,1],dtype='int')
All_valseg = np.empty([0,1],dtype='int')
All_trainseg = np.empty([0,1],dtype='int')
All_modelname = []
All_mod_name = []
count = np.empty([0,1],dtype='int'); #initialize model run -- this serves as the model run ID number
ct = 0
for a in test_model_nodes:
  for b in test_model_seq:
    count = np.append(count,  ct + 1 )
    #if statement for valseg, trainseg based on sequence length
    if int(b) == 699:
      trainseg = 2
      valseg = 1
   

    All_nodes = np.append(All_nodes, a) 
    All_seq = np.append(All_seq, int(b))
    All_valseg = np.append(All_valseg, valseg)
    All_trainseg = np.append(All_trainseg, trainseg)
    All_modelname = np.append(All_modelname, 'UNIT_' + str(a) + '_LB_' + str(b) + '_run_' + str(count[-1]) + '/' )
    All_mod_name = np.append(All_mod_name, 'UNIT_' + str(a) + '_LB_' + str(b) + '_run_' + str(count[-1]) )

    if ct+1 < runs:
      ct += 1
    else:
      ct = 0

print(All_mod_name)
j = 0
print(path + All_modelname[j] + All_mod_name[j] + '_bestwhole.h5') 

['UNIT_256_LB_699_run_1']
/home/victorphilippecodex/Documents/Hand_Signature/WholeGroup1UNIT_256_LB_699_run_1/UNIT_256_LB_699_run_1_bestwhole.h5


In [23]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [24]:
for j in range(len(All_mod_name)):
    # make folder to store model
    newfoldpath = path + All_mod_name[j]


    # extract path to store each model and generated data
    savepath = path + All_modelname[j]
    mod_name = All_mod_name[j]

    print('Working on: ' + mod_name + ' model ' + str(j) + ' / ' + str(len(All_mod_name)))
    
    # Specify variables for model run instance

    # Number of Units
    numunits = All_nodes[j]

    # Lookback parameter
    # A number is chosen called the look back parameter where training occurs for this many timesteps and then the units' weights are 
    #reset to zero. Within the look back parameter # of time steps the outputs are predicted, errors calculated, gradient attained which is multiplied by 
    #back propagation to update the weights.  If you add 1 to the lookback parameter- this number should be divisible by the trialsamp- this allows for
    # generating the input and the output data within each dataset- example if triallength is 30, the lookback parameter could be 9
    # so input1 = 1:9, output1 = 2:10.....input3= 21:29, output3= 22:30.Thus there are 3 times we can evaluate error within the trial (resetting paramters each time) 
    lookback = All_seq[j]

    # Training and Validation Set-up
    # Based on the length of the trials and the lookback parameter you can set how much data would be for training vs testing. 
    # For example, if the num = lookback+1 can be divided into the trial length 6 times,  
    # trainseg can be 4 corresponding to the 1st 4 segments and valseg can be 2 corresponding to last 2 segments of trial for validation
    # Select the 1st X segments to be training
    trainseg = All_trainseg[j]
    # Select the last Y segments to be validation
    valseg = All_valseg[j]

    with tf.device('/device:GPU:1'):
      # load modelfrom previous run to save time re-training model
      model = tf.keras.models.load_model(savepath + mod_name + '_bestwhole.h5') 

      # Need to be on GPU to run the following as well:

      # Set the trained model weights to new model to evaluate external predictions and self-driving 
      model2=tf.keras.models.Sequential()
      model2.add(tf.compat.v1.keras.layers.LSTM(units = numunits, stateful=True, return_sequences=True, batch_input_shape =(1,None,feats)))
      model2.add(tf.compat.v1.keras.layers.Dense(units=feats))
      model2.compile(loss='mse', optimizer='adam',metrics=['accuracy'])


      # Set model 2 weights from model 1
      for l1, l2 in zip(model.layers, model2.layers):
        assert l1.get_config().keys()==l2.get_config().keys()
      for key, v in l1.get_config().items():
          if key=='name':
              continue
          if not v==l2.get_config()[key]:
              print(l1.name, key, 'not matching.', 'Old,new : ', v, l2.get_config()[key], '\n')

      model2.set_weights(model.get_weights())
      assert np.all([np.array_equal(w1, w2) for (w1,w2) in zip(model.get_weights(), model2.get_weights())]) 

      ####  Generate external and self driving signatures for all data
      print('Running self-driven gait signatures')

      in_data = all_inputdata_s
      ext_drive= trialsamp # externally drive for twice the lookback
      total_drive = ext_drive*2  # in self-driving mode the lower node models won't self-drive more than the look-back- they zero out pretty quickly 

      HC_self_concat = np.empty(shape=[0, numunits*2]) # initialize HC storage for self-driving
      HC_ext_concat = np.empty(shape=[0, numunits*2])  # initialize HC storage for external-driving

      Predicted_KIN = np.empty(shape = [trialnum,total_drive, feats])
      
      test_ind = np.arange(0, len(all_inputdata_s), 1)  # run all input data through

      for g in test_ind:
          print('external and self-driven HCs parsed: ' + str(g) + ' of ' + str(len(test_ind)))
          model2.layers[0].reset_states([np.zeros((1,numunits)), np.zeros((1,numunits))])
          preds = np.zeros((1, total_drive, feats))
          hcs = np.zeros((2, total_drive, numunits))
          for i in tqdm(range(total_drive)):
              if i<ext_drive:
                  #preds[:,i,:] = model2.predict(in_data[g,i,:][None,None,:], use_multiprocessing = True)# index the test trial
                  preds[:,i,:] = model2.predict_step(in_data[g,i,:][None,None,:])# index the test trial
              else:
                  #preds[:,i,:] = model2.predict(preds[0,i-1,:][None,None,:])
                  preds[:,i,:] = model2.predict_step(preds[0,i-1,:][None,None,:])

              hcs[:,i,:] = np.array(model2.layers[0].states)[:,0]

              Hvalues = hcs[0]
              Cvalues = np.tanh(hcs[1])

              Hvalues_self = Hvalues[ext_drive:total_drive]# only extract the selfdriving Hs
              Hvalues_ext = Hvalues[0:ext_drive] # extract external Hs

              Cvalues_self = Cvalues[ext_drive:total_drive]
              Cvalues_ext = Cvalues[0:ext_drive]

              HC_self = np.concatenate((Hvalues_self, Cvalues_self), axis=1) # concatenate Hs and Cs of single trial
              HC_ext =  np.concatenate((Hvalues_ext, Cvalues_ext), axis=1)

          HC_self_concat = np.append(HC_self_concat, HC_self, axis = 0) # concatenate all trials to eachother
          HC_ext_concat = np.append(HC_ext_concat, HC_ext, axis = 0)
          
          preds_rescale = preds[0,:,:] # no rescaling in this script - unscaled model (if scaled it would be rescaled here)
          in_data_rescale = in_data[g] # no rescaling - unscaled model (if scaled it would be rescaled here)

          Predicted_KIN[g] = preds_rescale
          #np.save(savepath + mod_name + 'trial_' + str(g) + '_rescaledpredictedkinematics.npy', preds_rescale) # save the predicted kinematics
          #np.save(savepath + mod_name + 'trial_' + str(g) + '_rescaledkinematics.npy', in_data_rescale) #save the rescaled original kinematics for testin

          # Generate figure with predictive kinematics
          fig = plt.figure(figsize=(20,15))
          plt.subplot(15,1,1)
          plt.title("Trial: " + str(g), fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),0], "b-", markersize=21, label="Ind 17 Right Thumb X") #original data
          plt.plot(preds_rescale[:,0], "g-", markersize=20, label="Ind 17 Right Thumb X_predict") #predicted
          plt.axvline(x=ext_drive, color='r', alpha=0.4)
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")

          plt.subplot(15,1,2)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),1], "b-", markersize=21, label="Ind 17 Right Thumb Y") #original data
          plt.plot(preds_rescale[:,1], "g-", markersize=20, label="Ind 17 Right Thumb Y_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,3)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),2], "b-", markersize=21, label="Ind 17 Right Thumb Z") #original data
          plt.plot(preds_rescale[:,2], "g-", markersize=20, label="Ind 17 Right Thumb Z_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,4)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),3], "b-", markersize=21, label="Ind 17 Right Index X") #original data
          plt.plot(preds_rescale[:,3], "g-", markersize=20, label="Ind 17 Right Index X_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,5)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),4], "b-", markersize=21, label="Ind 17 Right Index Y") #original data
          plt.plot(preds_rescale[:,4], "g-", markersize=20, label="Ind 17 Right Index Y_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,6)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),5], "b-", markersize=21, label="Ind 17 Right Index Z") #original data
          plt.plot(preds_rescale[:,5], "g-", markersize=20, label="Ind 17 Right Index Z_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,7)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),6], "b-", markersize=21, label="Ind 17 Right Middle X") #original data
          plt.plot(preds_rescale[:,6], "g-", markersize=20, label="Ind 17 Right Middle X_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,8)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),7], "b-", markersize=21, label="Ind 17 Right Middle Y") #original data
          plt.plot(preds_rescale[:,7], "g-", markersize=20, label="Ind 17 Right Middle Y_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,9)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),8], "b-", markersize=21, label="Ind 17 Right Middle Z") #original data
          plt.plot(preds_rescale[:,8], "g-", markersize=20, label="Ind 17 Right Middle Z") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,10)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),9], "b-", markersize=21, label="Ind 17 Right Ring X") #original data
          plt.plot(preds_rescale[:,9], "g-", markersize=20, label="Ind 17 Right Ring X_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,11)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),10], "b-", markersize=21, label="Ind 17 Right Ring Y") #original data
          plt.plot(preds_rescale[:,10], "g-", markersize=20, label="Ind 17 Right Ring Y_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,12)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),11], "b-", markersize=21, label="Ind 17 Right Ring Z") #original data
          plt.plot(preds_rescale[:,11], "g-", markersize=20, label="Ind 17 Right Ring Z_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,13)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),12], "b-", markersize=21, label="Ind 17 Right Ring X") #original data
          plt.plot(preds_rescale[:,12], "g-", markersize=20, label="Ind 17 Right Ring X_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,14)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),13], "b-", markersize=21, label="Ind 17 Right Pinky Y") #original data
          plt.plot(preds_rescale[:,13], "g-", markersize=20, label="Ind 17 Right Pinky Y_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.subplot(15,1,15)
          #plt.title("Evaluating Predictions", fontsize=14)
          plt.plot(in_data_rescale[:ext_drive+(trialsamp-ext_drive),14], "b-", markersize=21, label="Ind 17 Right Pinky Z") #original data
          plt.plot(preds_rescale[:,14], "g-", markersize=20, label="Ind 17 Right Pinky Z_predict") #predicted
          plt.legend(loc="upper right")
          plt.xlabel("Sample Number")
          plt.ylabel("Position")
          plt.axvline(x=ext_drive, color='r', alpha=0.4)

          plt.savefig(savepath + mod_name + 'trial_' + str(g) +  '_PredictiveKinematics.png', dpi = 300)
          plt.close(fig)# close figure in loop

      np.save(savepath + mod_name + '_PredictedKinematics_ext_self.npy', Predicted_KIN)

      SelfDriveHCs_testset = HC_self_concat # rename for further processing already coded before
      ExternalDriveHCs_testset = HC_ext_concat

      CombinedExternal_Self_HCs = np.concatenate((ExternalDriveHCs_testset,SelfDriveHCs_testset), axis = 0) # all Hs and Cs

        # save these held out HCs 
      np.save(savepath + mod_name + '_selfdriveHCs.npy', SelfDriveHCs_testset)
      scipy.io.savemat(savepath + mod_name + '_selfdriveHCs.mat', {'self_drive_sigs': SelfDriveHCs_testset})

      np.save(savepath + mod_name + '_extdriveHCs_testset.npy', ExternalDriveHCs_testset)
      scipy.io.savemat(savepath + mod_name + '_extdriveHCs_testset.mat', {'ext_drive_sigs': ExternalDriveHCs_testset})


Working on: UNIT_256_LB_699_run_1 model 0 / 1
Running self-driven gait signatures
external and self-driven HCs parsed: 0 of 252


100%|██████████| 4200/4200 [01:38<00:00, 42.82it/s]


external and self-driven HCs parsed: 1 of 252


100%|██████████| 4200/4200 [01:37<00:00, 42.98it/s]


external and self-driven HCs parsed: 2 of 252


...

100%|██████████| 4200/4200 [01:39<00:00, 42.04it/s]


external and self-driven HCs parsed: 250 of 252


100%|██████████| 4200/4200 [01:40<00:00, 41.98it/s]


external and self-driven HCs parsed: 251 of 252


100%|██████████| 4200/4200 [01:40<00:00, 41.62it/s]
